In [72]:
from imutils.video import VideoStream
import numpy as np

import cv2
import imutils
import time

from imutils.video import FileVideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import cv2

path_vid = 'Material/Videos/full.mp4'
path_vid_small = 'Material/Videos/short.mp4'

In [7]:
# video = VideoStream(src=path_vid).start()
 
# # allow the camera or video file to warm up
# time.sleep(2.0)
# initial = True
# flag = False
# current_key_pressed = set()
# circle_radius = 30

In [12]:
fvs = FileVideoStream(path_vid).start()
time.sleep(1.0)

while True:
    frame = fvs.read()
    
    if frame is None:
        break
        
    frame = imutils.resize(frame, width=450)
    cv2.imshow('Vid', frame)
     
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
    
cv2.destroyAllWindows()

In [26]:
path_vid = 'Material/Videos/full.mp4'

In [23]:
cv2.destroyAllWindows()
fvs.stop()

In [27]:
def draw(ret, bbox, frame):
         # Draw bounding box
    if ret:
        # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
        # Tracking failure
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)


In [36]:
import ctypes
import time

SendInput = ctypes.windll.user32.SendInput


W = 0x11
A = 0x1E
S = 0x1F
D = 0x20

# C struct redefinitions 
PUL = ctypes.POINTER(ctypes.c_ulong)
class KeyBdInput(ctypes.Structure):
    _fields_ = [("wVk", ctypes.c_ushort),
                ("wScan", ctypes.c_ushort),
                ("dwFlags", ctypes.c_ulong),
                ("time", ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class HardwareInput(ctypes.Structure):
    _fields_ = [("uMsg", ctypes.c_ulong),
                ("wParamL", ctypes.c_short),
                ("wParamH", ctypes.c_ushort)]

class MouseInput(ctypes.Structure):
    _fields_ = [("dx", ctypes.c_long),
                ("dy", ctypes.c_long),
                ("mouseData", ctypes.c_ulong),
                ("dwFlags", ctypes.c_ulong),
                ("time",ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class Input_I(ctypes.Union):
    _fields_ = [("ki", KeyBdInput),
                 ("mi", MouseInput),
                 ("hi", HardwareInput)]

class Input(ctypes.Structure):
    _fields_ = [("type", ctypes.c_ulong),
                ("ii", Input_I)]

# Actuals Functions

def PressKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

def ReleaseKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008 | 0x0002, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

if __name__ == '__main__':
    PressKey(0x11)
    time.sleep(1)
    ReleaseKey(0x11)
    time.sleep(1)

In [77]:
def get_centroid(bbox):
    x, y, w, h = bbox
    centx = int(x + w // 2)
    centy = int(y + h // 2)
    return (centx, centy)

In [121]:
def action_key(key):
    PressKey(key)
    pass

pressed_s, pressed_a = None, None

def steer(bbox):
    #left
    global pressed_s
    x, _ = get_centroid(bbox)
    diff = x - centleft[0]
    
    if abs(diff) < THRESH_ACT:
        return
    
    if diff > 0:
        pressed_s = D
    else:
        pressed_s = A
        
    action_key(pressed_s)
    return diff    
    
def accelerate(bbox):
    global pressed_a
    
    _, y = get_centroid(bbox)
    diff = - (y - centleft[1])  #as y decr upward

    if abs(diff) < THRESH_ACT:
        return
    
    if diff > 0:
        pressed_a = W
    else:
        pressed_a = S
        
    action_key(pressed_a)
    
    return diff

RADIUS = 60
THRESH_ACT = 20#RADIUS

def drawc(frame, center, radius=RADIUS, color=COLOR_BLACK, thickness=3):
    cv2.circle(frame, center, radius, color, thickness)
    
# def release():
   
    

def action(bboxleft, bboxright):
    global pressed_s, pressed_a
    
    a=accelerate(bboxright)
    s=steer(bboxleft)
#     if a is None:
    if s is None and pressed_s is not None:
        ReleaseKey(pressed_s)
        pressed_s = None
    if a is None and pressed_a is not None:
        ReleaseKey(pressed_a)
        pressed_a = None
    return a, s

def releaseAll():
    for key in ['W', 'S', 'A', 'D']:
        ReleaseKey(key)

In [120]:
fvs = FileVideoStream(path_vid).start()
frame = fvs.read()
frame = cv2.flip(frame, 1)

trackerleft = cv2.TrackerCSRT_create()
trackerright = cv2.TrackerCSRT_create()

bboxleft = bbl#cv2.selectROI(frame, False)
bboxright = bbr#cv2.selectROI(frame, False)

centleft = get_centroid(bboxleft)
centright = get_centroid(bboxright)

drawc(frame, centleft)
drawc(frame, centright)

# Initialize tracker with first frame and bounding box
trackerleft.init(frame, bboxleft)
trackerright.init(frame, bboxright)

cv2.waitKey(1000)

cv2.destroyAllWindows()

while True:
    frame = fvs.read()
    
    if frame is None:
        break
        
    frame = cv2.flip(frame, 1)
#     frame = imutils.resize(frame, width=size_width)  

    retleft, bboxleft = trackerleft.update(frame)
    retright, bboxright = trackerright.update(frame)
    
    if not retleft == retright == True:
        print("Tracking stopped")
        break
    
    a,s  = action(bboxleft, bboxright)
    
    cv2.putText(frame, "a  "+str(pressed_a), (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
    cv2.putText(frame, "s  "+str(pressed_s), (100,90), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
     
#     draw(retleft, bboxleft, frame)
#     draw(retright, bboxright, frame)

    drawc(frame, centleft)
    drawc(frame, centright)

#     write('')

    cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK,2);
#     cv2.putText(frame, "FPSd : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
    cv2.imshow("Tracking", frame)

    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
    
releaseAll()
    
cv2.destroyAllWindows()
fvs.stop()

In [99]:
def write(s, val, frame):
    cv2.putText(frame, s, (100, 50+val), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);

In [116]:
cv2.destroyAllWindows()
fvs.stop()

In [68]:
COLOR_BLACK = (255,255,255)#(0,0,0)
COLOR_BLACK = (0,0,0)
COLOR_RED = (0, 0, 255)